## Modelo de Recomendacion

### Importamos las librerias necesarias para comenzar el proceso de ETL 

In [28]:
import pandas as pd
import numpy as np
import herramientas
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import operator


import pyarrow as pa
import pyarrow.parquet as pq

Se leen los datos del archivo .csv y se guardan en un dataframe

In [29]:
df_games=pd.read_csv(r'data\steam_games_limpio.csv')
df_items=pd.read_csv(r'data\user_items_limpio.csv')
df_items

,item_id,item_name,playtime_forever,user_id,items_count,steam_id
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479
...,...,...,...,...,...,...
5094077,346330,BrainBread 2,0,76561198329548331,7,76561198329548331
5094078,373330,All Is Dust,0,76561198329548331,7,76561198329548331
5094079,388490,One Way To Die: Steam Edition,3,76561198329548331,7,76561198329548331
5094080,521570,You Have 10 Seconds 2,4,76561198329548331,7,76561198329548331


Verificamos la existencia de nulos y los tipos de datos de las columnas de los dataframes 

In [30]:
herramientas.verifica_tipo_y_nulos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.00,0.00,0
1,price,[<class 'float'>],100.00,0.00,0
2,early_access,[<class 'bool'>],100.00,0.00,0
3,id,[<class 'int'>],100.00,0.00,0
4,release_anio,[<class 'str'>],100.00,0.00,0
5,publisher,"[<class 'str'>, <class 'float'>]",99.93,0.07,47
6,title,[<class 'str'>],100.00,0.00,0
7,developer,[<class 'str'>],100.00,0.00,0


In [31]:
herramientas.verifica_tipo_y_nulos(df_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'int'>],100.0,0.0,0
1,item_name,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,user_id,[<class 'str'>],100.0,0.0,0
4,items_count,[<class 'int'>],100.0,0.0,0
5,steam_id,[<class 'int'>],100.0,0.0,0


Verifico duplicados 

In [32]:
df_games_m=df_games[['developer','id', 'title']]
df_games_m

,developer,id,title
0,Kotoshiro,761140,Lost Summoner Kitty
1,Kotoshiro,761140,Lost Summoner Kitty
2,Kotoshiro,761140,Lost Summoner Kitty
3,Kotoshiro,761140,Lost Summoner Kitty
4,Kotoshiro,761140,Lost Summoner Kitty
...,...,...,...
71546,Laush Dmitriy Sergeevich,610660,Russian Roads
71547,Laush Dmitriy Sergeevich,610660,Russian Roads
71548,Laush Dmitriy Sergeevich,610660,Russian Roads
71549,"xropi,stev3ns",658870,EXIT 2 - Directions


Agrupo por titulo y aplico una función lambda a la columna ‘developer’. Esta función convierte cada entrada en la columna ‘developer’ a una cadena (si no lo es ya) y luego une todas las cadenas en el grupo con ', ’ entre ellas. Esto resulta en una única cadena que contiene todos los valores de ‘developer’ para cada título, separados por comas

In [33]:
#Se agrupa por título y concatena las entradas de la columna 'rating'
df_agrupado_con_developer = df_games_m.groupby('title')['developer'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

#Se visualiza la tabla
df_agrupado_con_developer 

,title,developer
0,! That Bastard Is Trying To Steal Our Gold !,"WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
1,"""Barely Attuned Magic Thingy"" Staff","Wild Shadow Studios, Wild Shadow Studios, Wild..."
2,"""Glow Ball"" - The billiard puzzle game","WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
3,"""Just Another Day"" - Seduce Me Otome CD","Michaela Laws, Michaela Laws"
4,"""Lethargic Sentience"" Wand","Wild Shadow Studios, Wild Shadow Studios, Wild..."
...,...,...
28823,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"YETU GAME, YETU GAME, YETU GAME, YETU GAME"
28824,４人打ちアクション麻雀 / ACTION MAHJONG,"Mindware Co.,Ltd., Mindware Co.,Ltd., Mindware..."
28825,＜/reality＞,"Fancy Fish Games, Fancy Fish Games"
28826,＜/reality＞ Original Soundtrack,"Fancy Fish Games, Fancy Fish Games"


Verifico el tipo de dato y si existen nulos 

In [34]:
herramientas.verifica_tipo_y_nulos(df_agrupado_con_developer)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,title,[<class 'str'>],100.0,0.0,0
1,developer,[<class 'str'>],100.0,0.0,0


Verifico la existencia de duplicados en todo el dataframe

In [35]:
df_agrupado_con_developer.duplicated().any()

False

Crea un objeto CountVectorizer, que es una herramienta para convertir una colección de documentos de texto en una matriz de conteos de tokens. 

- **max_features = 2000**: Limita el número de características (palabras únicas) que el vectorizador tomará en cuenta, manteniendo solo las 2000 palabras más frecuentes.
- **stop_words = "english"**: Le dice al vectorizador que ignore las palabras comunes en inglés (como ‘the’, ‘is’, ‘and’, etc.) que no aportan mucha información sobre el contenido del texto.

Luego, el método **fit_transform** se aplica a la columna “developer”. Este método aprende el vocabulario de la colección de documentos (en este caso, los nombres de los desarrolladores) y devuelve una matriz donde cada fila corresponde a un documento y cada columna a una palabra del vocabulario. Las entradas de la matriz son el número de veces que cada palabra aparece en cada documento.

El método toarray convierte la 'matriz dispersa' devuelta por **fit_transform** en una 'matriz densa' (un array de NumPy), y shape devuelve las dimensiones de esta matriz. La primera dimensión es el número de documentos (en este caso, el número de filas en df_agrupado_con_developer) y la segunda dimensión es el número de palabras en el vocabulario (hasta un máximo de 2000, como se especificó anteriormente).

In [36]:
cv = CountVectorizer(max_features = 2000, stop_words="english")
cv.fit_transform(df_agrupado_con_developer["developer"]).toarray().shape

(28828, 2000)

Este método **(vectors = cv.fit_transform(df_agrupado_con_developer["developer"]))** aprende el vocabulario de la colección de documentos (en este caso, los nombres de los desarrolladores) y devuelve una matriz donde cada fila corresponde a un documento y cada columna a una palabra del vocabulario. Las entradas de la matriz son el número de veces que cada palabra aparece en cada documento.

 - **.toarray()**: Convierte la matriz dispersa devuelta por fit_transform en una matriz densa (un array de NumPy). 



In [37]:
vectors = cv.fit_transform(df_agrupado_con_developer["developer"]).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Esta función **(cosine_similarity(vectors))** calcula la similitud del coseno entre cada par de vectores en la matriz vectors. En este caso guarda la matriz de similitud del coseno en la variable similarity

- **similarity[0]**: Muestra el primer elemento de la matriz de similitud del coseno, que es un vector que contiene las similitudes del coseno entre el primer vector en vectors y todos los demás vectores.


In [38]:
similarity = cosine_similarity(vectors)
similarity[0]

array([1., 0., 1., ..., 0., 0., 0.])

Por medio de la funcion **sorted** se obtendra una lista de las cinco tuplas que representan los cinco vectores más similares al primer vector, junto con sus índices originales y sus valores de similitud del coseno.

In [39]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(2, 1.0), (10529, 1.0), (12432, 1.0), (21017, 1.0), (1, 0.0)]

La siguiente funcion (recomendacion) toma como entrada el nombre de un juego y devuelve una lista de juegos recomendados basándose en la similitud de los desarrolladores.

In [40]:
def recomendacion(game):
    indice_juegos = df_agrupado_con_developer[df_agrupado_con_developer["title"]==game].index[0]
    distances = similarity[indice_juegos]
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_titles = [df_agrupado_con_developer.iloc[i[0]]['title'] for i in lista_juegos]
    return recommended_titles

Aplico la función recomendacion a la columna ‘title’ del dataframe df_agrupado_con_developer y almacenando los resultados en una nueva columna llamada ‘model’.

Luego de ejecutar este código, cada fila en df_agrupado_con_developer tendrá una nueva columna ‘model’ que contiene una lista de juegos recomendados para el título de esa fila.


In [41]:
df_agrupado_con_developer['model']=df_agrupado_con_developer['title'].apply(recomendacion)

Elimino  la columna 'developer '

In [42]:
df_agrupado_con_developer_del=df_agrupado_con_developer.drop('developer', axis=1, inplace= True)

Creamos un dataframe el cual nos traiga las columas 'id', y 'title' del dataframe df_games. Luego verificamos los valores duplicados del mismo

In [43]:
df_games_m_m=df_games_m[['id','title']]
df_games_clean=df_games_m_m.drop_duplicates(subset='title',keep='first')
df_games_clean

,id,title
0,761140,Lost Summoner Kitty
5,643980,Ironbound
9,670290,Real Pool 3D - Poolians
14,767400,弹炸人2222
17,772540,Battle Royale Trainer
...,...,...
71535,745400,Kebab it Up!
71539,773640,Colony On Mars
71543,733530,LOGistICAL: South Africa
71546,610660,Russian Roads


Verifico la existencia de los duplicados de la columna 'title'

In [44]:
herramientas.verifica_duplicados_por_columna(df_games_clean,'title')

'No hay duplicados'

Verifico la existencia de valores nulos y el tipo de dato 

In [45]:
herramientas.verifica_tipo_y_nulos(df_games_clean)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,id,[<class 'int'>],100.0,0.0,0
1,title,[<class 'str'>],100.0,0.0,0


Realizo un merge entre los dataframes, por la columna en comun 'title'

In [46]:
df_merge=pd.merge(df_agrupado_con_developer,df_games_clean, on='title', how='inner')
df_merge

,title,model,id
0,! That Bastard Is Trying To Steal Our Gold !,"[""Glow Ball"" - The billiard puzzle game, Gold ...",449940
1,"""Barely Attuned Magic Thingy"" Staff","[""Lethargic Sentience"" Wand, ""Precisely Calibr...",308163
2,"""Glow Ball"" - The billiard puzzle game","[""Glow Ball"" - The billiard puzzle game, Gold ...",388390
3,"""Just Another Day"" - Seduce Me Otome CD","[My Lady, Remember, Remember, ""Just Another Da...",454790
4,"""Lethargic Sentience"" Wand","[""Lethargic Sentience"" Wand, ""Precisely Calibr...",308164
...,...,...,...
28823,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"[PRICE - Original Soundtrack（原声OST）, PRICE Des...",541220
28824,４人打ちアクション麻雀 / ACTION MAHJONG,"[HEIANKYO ALIEN / 平安京エイリアン, Pinball Parlor, SP...",575810
28825,＜/reality＞,"[ADventure Lib Original Soundtrack, Deity Ques...",562280
28826,＜/reality＞ Original Soundtrack,"[ADventure Lib Original Soundtrack, Deity Ques...",626850


Verifico la existencia de nulos 

In [47]:
herramientas.verifica_tipo_y_nulos(df_merge)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,title,[<class 'str'>],100.0,0.0,0
1,model,[<class 'list'>],100.0,0.0,0
2,id,[<class 'int'>],100.0,0.0,0


Del nuevo dataframe extraigo solo las columnas 'id' y 'model'

In [48]:
df_merge_id=df_merge[['id','model']]
df_merge_id

,id,model
0,449940,"[""Glow Ball"" - The billiard puzzle game, Gold ..."
1,308163,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."
2,388390,"[""Glow Ball"" - The billiard puzzle game, Gold ..."
3,454790,"[My Lady, Remember, Remember, ""Just Another Da..."
4,308164,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."
...,...,...
28823,541220,"[PRICE - Original Soundtrack（原声OST）, PRICE Des..."
28824,575810,"[HEIANKYO ALIEN / 平安京エイリアン, Pinball Parlor, SP..."
28825,562280,"[ADventure Lib Original Soundtrack, Deity Ques..."
28826,626850,"[ADventure Lib Original Soundtrack, Deity Ques..."


Guardamos el ultimo dataframe en un archivo parquet, el cual utilizaremos para realizar la consulta

In [49]:
#Guarda el DataFrame en el archivo PARQUET
df_merge_id.to_parquet('data/df_recomendacion.parquet')

print(f'Se guardó el archivo {df_merge_id}')

Se guardó el archivo            id                                              model
0      449940  ["Glow Ball" - The billiard puzzle game, Gold ...
1      308163  ["Lethargic Sentience" Wand, "Precisely Calibr...
2      388390  ["Glow Ball" - The billiard puzzle game, Gold ...
3      454790  [My Lady, Remember, Remember, "Just Another Da...
4      308164  ["Lethargic Sentience" Wand, "Precisely Calibr...
...       ...                                                ...
28823  541220  [PRICE - Original Soundtrack（原声OST）, PRICE Des...
28824  575810  [HEIANKYO ALIEN / 平安京エイリアン, Pinball Parlor, SP...
28825  562280  [ADventure Lib Original Soundtrack, Deity Ques...
28826  626850  [ADventure Lib Original Soundtrack, Deity Ques...
28827  460250  [Invisible Apartment, Invisible Apartment - So...

[28828 rows x 2 columns]


- **Funcion: recomendacion_juego (id)**

La función recomendacion_juego(id:int) Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [50]:
def recomendacion_juego(id:int):
    # Filtra el DataFrame df_merge_id por el id proporcionado y obtiene el 'model' correspondiente
    modelo = df_merge_id[df_merge_id['id'] == id]['model'].iloc[0]
    
    # Inicializa un diccionario vacío para almacenar las recomendaciones
    recomendaciones_dict = {}
    
    # Si el modelo tiene elementos, procede a llenar el diccionario de recomendaciones
    if len(modelo) > 0:
        for i in range(len(modelo)):
            # Agrega cada elemento del modelo al diccionario de recomendaciones
            recomendaciones_dict[i + 1] = modelo[i]
        # Devuelve el diccionario de recomendaciones
        return recomendaciones_dict
    else:
        # Si el modelo no tiene elementos, devuelve un mensaje de error
        return f"No se encontró un modelo para el id {id}"


In [51]:
recomendacion_juego(454790)

{1: 'My Lady',
 2: 'Remember, Remember',
 3: '"Just Another Day" - Seduce Me Otome CD',
 4: 'Seduce Me 2: The Demon War',
 5: 'Seduce Me the Otome'}

Verifico la existencia de alguno de los juegos dentro del dataframe para verificar el correcto funcionamiento del modelo 

In [52]:
df=df_merge[df_merge['title']=="My Lady"]
df

,title,model,id
15294,My Lady,"[My Lady, Remember, Remember, ""Just Another Da...",457530
